## Moving Average for Forex

In [ ]:
class MACrossover(QCAlgorithm):

    def Initialize(self):
        self.SetStartDate(2015, 1, 1)  # Set Start Date
        self.SetCash(10000)  # Set Strategy Cash
   
        self.long_list =[]
        self.short_list =[]
        
        self.stop =False
        
        # Selection of these Forex pairs is based on Pearson correlation coefficient 
        # Reference: https://www.myfxbook.com/en/forex-market/correlation
        self.currencies = ["EURUSD", "GBPUSD", "USDCHF", "USDJPY", "EURJPY", "USDCAD", "AUDUSD"] 
        
        self.AddForex("EURUSD", Resolution.Daily)   
        self.AddForex("GBPUSD", Resolution.Daily)   
        self.AddForex("USDCHF", Resolution.Daily)    
        self.AddForex("USDJPY", Resolution.Daily)   
        self.AddForex("EURJPY", Resolution.Daily)   
        self.AddForex("USDCAD", Resolution.Daily) 
        self.AddForex("AUDUSD", Resolution.Daily)   


    def OnData(self, data):
        
        if self.stop:
            return
        
        currencies = self.currencies
        
        for currency in currencies:
            self.Debug(currency)
            currency_data = self.History ([currency], 30, Resolution.Daily)
            MA21_Pre = currency_data.close[7:30].mean()
            MA5_Pre = currency_data.close[25:30].mean()
            
            MA21_Current = currency_data.close[8:31].mean()
            MA5_Current = currency_data.close[26:31].mean()
            
            if currency not in self.long_list and currency not in self.short_list:
                print ("found currency" +str(currency))
                if MA5_Pre < MA21_Pre and MA5_Current > MA21_Current:
                    self.SetHoldings (currency, 0.5)
                    self.long_list.append(currency)
                    
                if MA5_Pre > MA21_Pre and MA5_Current < MA21_Current:
                    self.SetHoldings (currency, -0.5)
                    self.short_list.append(currency)
                    
            if currency in self.long_list:
                if MA5_Pre > MA21_Pre and MA5_Current < MA21_Current:
                    self.SetHoldings (currency, -0.5)
                    self.long_list.remove(currency)
                    self.short_list.append(currency)
                    
            if currency in self.short_list:
                 if MA5_Pre < MA21_Pre and MA5_Current > MA21_Current:
                    self.SetHoldings (currency, 0.5)
                    self.short_list.remove(currency)
                    self.long_list.append(currency)
                    
            if self.Portfolio.Cash < 0.85*10000:
                self.stop = True
                self.Liquidate()